In [1]:
import torch
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
def get_logits(sentence, target):

    tokens = torch.tensor(gpt2_tokenizer.encode(sentence)).to(device)
    outputs = gpt2_model(tokens, labels=tokens)
    logits = outputs["logits"][-1]
    id_pred = gpt2_tokenizer.encode(target)[0]
    logit_pred = logits[id_pred].item()

    return logit_pred


In [8]:
input_path = 'data.csv'
df = pd.read_csv(input_path, delimiter=';')


In [9]:
def process_pair(row):

    logit_aae = get_logits(row['aae'], row['pred'])
    logit_sae = get_logits(row['sae'], row['pred'])

    result = 1 if logit_sae > logit_aae else 0

    return result


In [ ]:
df['result'] = df.apply(process_pair, axis=1)

output_path = 'gpt-results.csv'
df.to_csv(output_path, sep=';', index=False)
